In [1]:
# -*- coding: utf-8 -*-
"""
---------------------------------------------------------------------------------------------------
 How to Set Up Direct Access the LP DAAC Data Pool with Python
 The following Python code will configure a netrc profile that will allow users to download data
 from an Earthdata Login enabled server. See README for additional information
---------------------------------------------------------------------------------------------------
 Author: Cole Krehbiel
 Last Updated: 11/20/2018
-------------------------------------------------------------------------------
"""
# Load necessary packages into Python
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import os

# -----------------------------------AUTHENTICATION CONFIGURATION-------------------------------- #
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

In [2]:
import ast
import os
import csv
from pandas import read_csv
from datetime import datetime
import fiona
import requests as r
import numpy as np
import pandas as pd
import getpass
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import json
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from pathlib import Path
import pickle
import pyproj
from pyproj import Proj
import shapely.geometry
from shapely.geometry import shape
from shapely.geometry.polygon import orient
from shapely.ops import transform
import xarray as xr
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')
# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
W = "%s-W%s" % (2015, 45)     

import datetime

## NASS week is ISO week
StartDate = datetime.datetime.strptime(W + '-1', "%G-W%V-%u")
EndDate = StartDate + datetime.timedelta(days=6)

EndDate

In [3]:
# Get json response from CMR collection metadata

short_name = 'NSIDC-0779'

params = {
    'short_name': 'NSIDC-0779'
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = r.get(cmr_collections_url, params=params)
results = json.loads(response.content)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)

The most recent version of  NSIDC-0779  is  1


In [ ]:
## transfrom the CRS of JSON to 6933
#import geopandas

#for filename in CSlist:
    
#data = geopandas.read_file(path+file_in)
# change CRS to epsg 4326
#data = data.to_crs(epsg=4326)
# write shp file
#data.to_file(path+file_out)

In [4]:
bounding_box = '-91.514727,36.970415,-87.494718,42.508773'

In [5]:
temporal = "2015-05-11T00:00:00Z/2015-11-08T23:59:59Z"    # Define start time period / end time period

In [6]:
# Create CMR parameters used for granule search. Modify params depending on bounding_box or polygon input.

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

# bounding box input:
search_params = {
'short_name': short_name,
'version': latest_version,
'temporal': temporal,
'page_size': 100,
'page_num': 1,
'bounding_box': bounding_box
}
    
granules = []
headers={'Accept': 'application/json'}
while True:
    response = r.get(granule_search_url, params=search_params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    search_params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')


There are 182 granules of NSIDC-0779 version 1 over my area and time of interest.


In [7]:
# GDAL configurations used to successfully access NSIDC-0779 via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')

In [8]:
directory = directory='.../Illisoy/Illinois_shp/county_shape_files/ToJSON'

county96df = read_csv('.../Illisoy/ALLRUN/96points.csv')

CSlist = county96df.County.unique()


In [10]:
len(CSlist)

96

In [11]:
directory = '.../Illisoy/Illinois_shp/county_shape_files/ToJSON/SMAPTransJSON'

allcountydate = []

direc = '.../Illisoy/ALLRUN'

datelist = []

for g in granules:

    datelist.append(g['time_start'].split('T')[0])
    
for filename in CSlist: 
        
    for g in granules:  
        
        try:

            with fiona.open(os.path.join(directory, filename + '.json'), "r") as shapefile:
                shapes = [feature["geometry"] for feature in shapefile]
            with rio.open(g['links'][0]['href']) as src:
                out_image, transformed = rio.mask.mask(src, shapes, crop=True, filled=True)
                out_meta = src.meta

            smvec = {'County': filename, 
                     'Date': g['time_start'].split('T')[0], 
                     'Band1': np.nanmean(out_image[0]),
                     'Band2': np.nanmean(out_image[1])}

            allcountydate.append(smvec)
            
        except:

            continue

            a = np.array(datelist) 

            b = np.array(pd.DataFrame(allcountydate).Date)

            while len(list((set(a) ^ set(b)))) > 1:

                try:

                    with fiona.open(os.path.join(directory, filename + '.json'), "r") as shapefile:
                        shapes = [feature["geometry"] for feature in shapefile]
                    with rio.open(g['links'][0]['href']) as src:
                        out_image, transformed = rio.mask.mask(src, shapes, crop=True, filled=True)
                        out_meta = src.meta

                    smvec = {'County': filename, 
                             'Date': g['time_start'].split('T')[0], 
                             'Band1': np.nanmean(out_image[0]),
                             'Band2': np.nanmean(out_image[1])}

                    allcountydate.append(smvec)

                except:

                    continue

                    a = np.array(datelist) 

                    b = np.array(pd.DataFrame(allcountydate).Date) 


In [12]:
smvec = ['County', 'Date', 'Band1', 'Band2']

smdict = {n : n for n in smvec}

direc = '.../Illisoy/ALLRUN'

with open(os.path.join(direc, '2015SM.csv'), 'w') as f:  
    w = csv.DictWriter(f, smdict.keys())
    w.writeheader()
    w.writerows(allcountydate)